In [1]:
import numpy as np
import pandas as pd

In [4]:
!python processing_dataset.py

X shape is (69623, 46) 
Y shape is (69623, 1)


In [5]:
!ls

 processing_dataset.py	'Project1.2-Python-Code '   x_test.csv	  y_test.csv
 proj1.2.ipynb		 Querylevelnorm.txt	    x_train.csv   y_train.csv


In [108]:
Xtr = pd.read_csv('./x_train.csv', index_col=0).values
ytr = pd.read_csv('./y_train.csv', index_col=0).values.ravel()

Xts = pd.read_csv('./x_test.csv', index_col=0).values
yts = pd.read_csv('./y_test.csv', index_col=0).values.ravel()

In [126]:
n_features = Xtr.shape[1]

### gaussian RBF

In [216]:
a = np.arange(9).reshape(3,3)

In [221]:
def fi(mu, sigma, vectorized=False):
    def f(x):
        return np.exp(
            -0.5 * (x-mu).T.dot(
                np.dot(np.linalg.inv(sigma),(x-mu))
            )
        )
    def f_vec(x):
        return np.exp(
            -0.5* np.sum(
                (x-mu).dot(
                np.linalg.inv(sigma)) * (x-mu),
                axis=1)
            )
    return f_vec if vectorized else f

In [225]:
M = [(np.arange(n_features) == i).astype('uint8')+np.random.uniform(0,0.5,size=n_features) for i in range(n_features)]

sigmas = [np.diag(m) for m in M] + [np.eye(n_features)]

In [232]:
mu = np.zeros(n_features)

In [233]:
fi_j = [fi(mu, S, vectorized=True) for S in sigmas]

In [238]:
Fi = np.matrix([f(Xtr) for f in fi_j]).T

In [240]:
Fi.shape

(55698, 47)

In [246]:
ytr = np.matrix(ytr).T

In [296]:
inv = np.linalg.inv

In [299]:
W = inv(Fi.T @ Fi) @ Fi.T @ ytr

In [288]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as MSE

In [302]:
lr = LinearRegression(n_jobs=-1, fit_intercept=False)

In [303]:
lr.fit(Fi, ytr)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=-1, normalize=False)

In [313]:
MSE(ytr, lr.predict(Fi))

0.39037365887042347

In [314]:
MSE(ytr, Fi @ W)

0.39037365887042347